# Notebook Setup

In [1]:
if 'google.colab' in str(get_ipython()):
  IN_COLLAB = True
else:
  IN_COLLAB = False

#TODO: CHANGE THIS BASED ON YOUR OWN LOCAL SETTINGS
MY_HOME_ABS_PATH = "/content/drive/MyDrive/W210/co2-flux-hourly-gpp-modeling"

In [2]:
if IN_COLLAB:
  from google.colab import drive
  drive.mount('/content/drive/')

Mounted at /content/drive/


## Import Modules

In [3]:
# install required modules quietly
required_packages = ['geopandas', 'pyspark', 'azure-storage-blob']

for p in required_packages: 
  try:
      __import__(p)
  except ImportError:
      %pip install {p} --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 10.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 

In [4]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import math
import json

import pyspark.pandas as pd
from calendar import monthrange
from datetime import datetime
from io import BytesIO

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Load locale custome modules
import sys
if IN_COLLAB:
  os.chdir(MY_HOME_ABS_PATH)
  sys.path.insert(0,os.path.abspath("./code/src/tools"))
else:
  sys.path.append(os.path.abspath("./code/src/tools"))

from CloudIO.AzStorageClient import AzStorageClient
from edahelpers import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [5]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

## Define Local Files System Constants

In [6]:
root_dir =  MY_HOME_ABS_PATH
tmp_dir =  root_dir + os.sep + '.tmp'
raw_data_dir = tmp_dir
data_dir = root_dir + os.sep + 'data'
cred_dir = root_dir + os.sep + '.cred'
az_cred_file = cred_dir + os.sep + 'azblobcred.json'

if IN_COLLAB:
  raw_data_dir = "/content/drive/MyDrive/CO2_flux_gpp_modeling/DS_capstone_23Spring_CO2/Data/half_hourly_data"

site_metadata_filename = data_dir + os.sep + 'site-metadata.csv'

# Output
tag = "0"
container = "test-sites-data"
blob_name = f"test-sites-data_v_{tag}.parquet" #Advisor suggested features only

In [7]:
# Selected Test Sites
test_sites = ["US-GLE", # ENF, Cold
              "US-AR1", # GRA, Temperate
              "US-Seg", # GRA, Arid
              "US-FR2", # WSA, Temperate
              "ES-LM2", # WSA, Arid
              "CA-Cbo", # DBF, Cold
              "FR-Lam", # CRO, Temperate
              "IT-Cpz", # EBF, Temperate
              "CN-Cha", # MF Cold
              "IT-Lsn", # OSH, Temperate
              ]

# Get Selected Site Metadata

In [8]:
# Load site metadata
included_site_features = ['site_id', 'filename', 'elevation', 'lat', 'long',
                          'koppen_sub', 'koppen_main', 'koppen_name', 'koppen_main_name',
                          'c3c4', 'c4_percent']
site_metadata_df = pd.read_csv(site_metadata_filename, usecols = included_site_features)

# only focus on target sites
site_metadata_df= site_metadata_df.loc[site_metadata_df['site_id'].isin(test_sites)]
print(f"size:{site_metadata_df.shape}")
site_metadata_df.reset_index(inplace=True, drop=True)
site_metadata_df

size:(10, 11)


,site_id,elevation,lat,long,koppen_sub,koppen_main,koppen_name,koppen_main_name,c3c4,c4_percent,filename
0,CN-Cha,NaN,42.40250,128.09580,22,4,Dwb,Cold,C3,12.17000,data_full_half_hourly_raw_v0_1_CN-Cha.csv
1,IT-Cpz,68.00000,41.70525,12.37611,8,3,Csa,Temperate,C3,0.00000,data_full_half_hourly_raw_v0_1_IT-Cpz.csv
2,US-GLE,3197.00000,41.36653,-106.23990,27,4,Dfc,Cold,C3,0.16000,data_full_half_hourly_raw_v0_1_US-GLE.csv
3,CA-Cbo,120.00000,44.31670,-79.93330,26,4,Dfb,Cold,C3,1.09000,data_full_half_hourly_raw_v0_1_CA-Cbo.csv
4,US-AR1,611.00000,36.42670,-99.42000,14,3,Cfa,Temperate,C4,27.54000,data_full_half_hourly_raw_v0_1_US-AR1.csv
5,US-FR2,271.90000,29.94950,-97.99620,14,3,Cfa,Temperate,C3,18.37000,data_full_half_hourly_raw_v0_1_US-FR2.csv
6,US-Seg,1622.00000,34.36230,-106.70190,5,2,BWk,Arid,mix,0.24000,data_full_half_hourly_raw_v0_1_US-Seg.csv
7,ES-LM2,270.00000,39.93459,-5.77588,7,2,BSk,Arid,C3,1.30000,data_full_half_hourly_raw_v0_1_ES-LM2.csv
8,FR-Lam,180.00000,43.49644,1.23788,14,3,Cfa,Temperate,rotation,3.10000,data_full_half_hourly_raw_v0_1_FR-Lam.csv
9,IT-Lsn,1.00000,45.74048,12.75030,14,3,Cfa,Temperate,C3,2.70000,data_full_half_hourly_raw_v0_1_IT-Lsn.csv


# Run Data Pipeline on Gold Sample Sites

In [9]:
all_features = ['TIMESTAMP_START', 'TIMESTAMP_END', 'TA_F', 'TA_F_QC', 'TA_ERA',
       'SW_IN_POT', 'SW_IN_F', 'SW_IN_F_QC', 'SW_IN_ERA', 'LW_IN_F',
       'LW_IN_F_QC', 'LW_IN_ERA', 'VPD_F', 'VPD_F_QC', 'VPD_ERA', 'P_F',
       'P_F_QC', 'P_ERA', 'PA_F', 'PA_F_QC', 'PA_ERA', 'NETRAD', 'PPFD_IN',
       'G_F_MDS', 'G_F_MDS_QC', 'LE_F_MDS', 'LE_F_MDS_QC', 'LE_CORR',
       'H_F_MDS', 'H_F_MDS_QC', 'H_CORR', 'NEE_VUT_REF', 'NEE_VUT_REF_QC',
       'NEE_CUT_REF', 'NEE_CUT_REF_QC', 'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF',
       'GPP_NT_CUT_REF', 'GPP_DT_CUT_REF', 'RECO_NT_VUT_REF',
       'RECO_DT_VUT_REF', 'RECO_NT_CUT_REF', 'RECO_DT_CUT_REF', 'datetime',
       'year', 'month', 'day', 'hour', 'SITE_ID', 'date', 'NEE_VUT_REF_qa',
       'SW_DIF', 'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
       'b7', 'IGBP', 'koppen']

In [10]:
def data_cleanup(site_id_file_df, target, target_qc, features):

    data_df = None
    # qc_flag_dtype = CategoricalDtype([0, 1, 2, 3], ordered=True)
    qc_flags_features = [s for s in included_features if "_QC" in s]

    # Iterate through each site:
    for i, r in site_id_file_df.iterrows():        
        if not r.filename or type(r.filename) != type(""):
            print(f'\nERROR: {r.site_id} is mssing hourly data.')
            continue

        # Get only `included_features` from file
        local_filename = raw_data_dir + os.sep + r.filename
        site_df = pd.read_csv(local_filename, usecols = included_features +  target_variable + target_variable_qc)
        site_df['datetime'] = pd.to_datetime(site_df['datetime'])
        site_df['date'] = pd.to_datetime(site_df['date'])
        site_df['minute'] = site_df['datetime'].dt.minute
        if len(qc_flags_features) != 0:
            site_df[qc_flags_features] = site_df[qc_flags_features].astype('int')
        site_df['site_id'] = r.site_id

        # Remove zero or negative SW
        site_df.drop(site_df[site_df['SW_IN_ERA'] <= 0].index, inplace = True)

        # Drop rows with NAs for Target Variable
        site_df.dropna(subset=target_variable, axis=0, inplace=True)

        # Drop rows with bad NEE_VUT_REF_QC (aka bad GPP records)
        site_df.drop(site_df[site_df[target_variable_qc[0]] == 3].index, inplace = True)
        site_df.drop(target_variable_qc, axis=1, inplace=True)

        # Drop rows with any NA
        site_df.dropna(axis=0, inplace=True)

        print(f"{r.site_id}: {site_df.shape}")
        if type(data_df) == type(None):
            data_df = site_df
        else:
            data_df = pd.concat([data_df, site_df])
            
    return data_df

def merg_site_metadata(data_df, site_metadata_df):
    # Merge with Site Metadata
    data_df = data_df.merge(site_metadata_df, how='left', left_on='site_id', right_on='site_id')
    return data_df

In [11]:
included_features = ['TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA',
                     'datetime', 'year', 'month', 'day', 'hour', 'date',
                     'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 
                     'IGBP', 'koppen']
target_variable_qc = ['NEE_VUT_REF_QC']
target_variable = ['GPP_NT_VUT_REF']

# Get Train Dataset
data_df = data_cleanup(site_metadata_df[['site_id','filename']],
                  target_variable, target_variable_qc,
                  included_features)
print(f"Data size after cleanup: {data_df.shape}")

data_df = merg_site_metadata(data_df, site_metadata_df.drop(['filename'], axis=1))
print(f"Data size after after merged with site metadata: {data_df.shape}")
display(data_df.head())


CN-Cha: (16228, 27)
IT-Cpz: (59175, 27)
US-GLE: (54687, 27)
CA-Cbo: (79273, 27)
US-AR1: (28956, 27)
US-FR2: (30426, 27)
US-Seg: (91884, 27)
ES-LM2: (58806, 27)
FR-Lam: (115812, 27)
IT-Lsn: (40182, 27)
Data size after cleanup: (575429, 27)
Data size after after merged with site metadata: (575429, 36)


,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,GPP_NT_VUT_REF,datetime,year,month,day,hour,date,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,elevation,lat,long,koppen_sub,koppen_main,koppen_name,koppen_main_name,c3c4,c4_percent
0,-4.69100,29.00900,278.69700,1.17100,0.06400,94.14900,0.13530,2003-03-13 06:00:00,2003,3,13,6,2003-03-13,0.10572,0.15259,0.03020,0.14550,0.19790,0.11090,0.14480,0.15500,0.10100,0.06580,MF,Cold,0,CN-Cha,NaN,42.40250,128.09580,22,4,Dwb,Cold,C3,12.17000
1,-4.48000,62.79600,278.69700,1.27900,0.06400,94.16300,-0.47300,2003-03-13 06:30:00,2003,3,13,6,2003-03-13,0.10572,0.15259,0.03020,0.14550,0.19790,0.11090,0.14480,0.15500,0.10100,0.06580,MF,Cold,30,CN-Cha,NaN,42.40250,128.09580,22,4,Dwb,Cold,C3,12.17000
2,-4.26900,95.27300,278.69700,1.38800,0.00000,94.17700,-0.46365,2003-03-13 07:00:00,2003,3,13,7,2003-03-13,0.10572,0.15259,0.03020,0.14550,0.19790,0.11090,0.14480,0.15500,0.10100,0.06580,MF,Cold,0,CN-Cha,NaN,42.40250,128.09580,22,4,Dwb,Cold,C3,12.17000
3,-4.05900,125.88600,278.69700,1.49600,0.00000,94.19100,-0.43926,2003-03-13 07:30:00,2003,3,13,7,2003-03-13,0.10572,0.15259,0.03020,0.14550,0.19790,0.11090,0.14480,0.15500,0.10100,0.06580,MF,Cold,30,CN-Cha,NaN,42.40250,128.09580,22,4,Dwb,Cold,C3,12.17000
4,-3.84800,385.60700,226.91400,1.60400,0.00000,94.20500,1.29680,2003-03-13 08:00:00,2003,3,13,8,2003-03-13,0.10572,0.15259,0.03020,0.14550,0.19790,0.11090,0.14480,0.15500,0.10100,0.06580,MF,Cold,0,CN-Cha,NaN,42.40250,128.09580,22,4,Dwb,Cold,C3,12.17000


In [12]:
data_df.describe()

,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,GPP_NT_VUT_REF,year,month,day,hour,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,minute,elevation,lat,long,koppen_sub,koppen_main,c4_percent
count,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000,559201.00000,575429.00000,575429.00000,575429.00000,575429.00000,575429.00000
mean,15.87811,371.66242,327.43647,10.00838,0.04451,93.77548,5.96719,2010.85302,6.60275,15.77924,11.97021,0.30026,0.50196,0.13399,0.09710,0.26551,0.05552,0.08818,0.28358,0.22491,0.14341,15.00571,715.54907,40.45408,-42.89669,14.34476,2.99913,3.84936
std,9.45291,263.22687,53.82894,9.50595,0.21210,9.79924,7.88720,4.92395,3.03538,8.82158,3.71476,0.13962,0.22899,0.07661,0.07166,0.06554,0.05709,0.05800,0.06519,0.09441,0.09571,15.00001,979.83832,4.33831,58.56188,7.69604,0.72310,6.93971
min,-30.68700,0.00100,107.63900,0.00000,0.00000,66.03100,-49.38330,2001.00000,1.00000,1.00000,4.00000,-0.09768,-0.02273,-0.01764,0.01480,0.08400,0.00500,0.01930,0.09130,0.02990,0.01230,0.00000,1.00000,29.94950,-106.70190,5.00000,2.00000,0.00000
25%,9.67400,136.92100,288.87800,3.34400,0.00000,84.93500,0.44793,2007.00000,4.00000,8.00000,9.00000,0.19508,0.33521,0.07072,0.04580,0.21390,0.02470,0.05100,0.23700,0.15580,0.06970,0.00000,120.00000,36.42670,-106.23990,7.00000,2.00000,0.24000
50%,16.30200,339.35900,333.21600,6.94900,0.00000,98.66800,2.98209,2011.00000,7.00000,16.00000,12.00000,0.29302,0.52421,0.12272,0.07550,0.26550,0.03890,0.07310,0.28850,0.20740,0.11470,30.00000,180.00000,41.70525,-5.77588,14.00000,3.00000,1.30000
75%,22.61600,578.55900,368.87500,13.45900,0.00000,99.82000,9.46656,2015.00000,9.00000,23.00000,15.00000,0.39185,0.69042,0.18176,0.12780,0.31120,0.06350,0.10360,0.33050,0.29220,0.18850,30.00000,1622.00000,43.49644,1.23788,22.00000,4.00000,3.10000
max,44.08700,1050.62600,473.08500,80.09100,13.03500,103.92100,83.61300,2020.00000,12.00000,31.00000,23.00000,1.41056,0.90943,0.38458,0.81200,0.77620,0.76400,0.80530,0.44180,0.47920,0.43470,30.00000,3197.00000,45.74048,128.09580,27.00000,4.00000,27.54000


In [13]:
data_df.site_id.unique()

array(['CN-Cha', 'IT-Cpz', 'US-GLE', 'CA-Cbo', 'US-AR1', 'US-FR2',
       'US-Seg', 'ES-LM2', 'FR-Lam', 'IT-Lsn'], dtype=object)

## Should be no NA data

In [14]:
total_record_count = data_df.shape[0]
na_df = pd.DataFrame(data_df.isna().sum())
na_df["percentage"] = (na_df / total_record_count)
na_df.rename(columns={0:"count"}, inplace=True)

In [15]:
na_df.loc[(na_df['count'] != 0)].sort_values("percentage", ascending=False)

,count,percentage
elevation,16228,0.02820


In [16]:
# Drop rows with NA
data_df.dropna(axis=0, inplace=True)
print(f"Data size after after final drop: {data_df.shape}")

Data size after after final drop: (559201, 36)


In [17]:
data_df.isna().sum()

TA_ERA              0
SW_IN_ERA           0
LW_IN_ERA           0
VPD_ERA             0
P_ERA               0
PA_ERA              0
GPP_NT_VUT_REF      0
datetime            0
year                0
month               0
day                 0
hour                0
date                0
EVI                 0
NDVI                0
NIRv                0
b1                  0
b2                  0
b3                  0
b4                  0
b5                  0
b6                  0
b7                  0
IGBP                0
koppen              0
minute              0
site_id             0
elevation           0
lat                 0
long                0
koppen_sub          0
koppen_main         0
koppen_name         0
koppen_main_name    0
c3c4                0
c4_percent          0
dtype: int64

# Upload Data to Azure Storage Blob as Parquet
**Run with Caution!!!**

In [18]:
# Upload to Azure Storage Blob
# ref: https://stackoverflow.com/a/54666079
parquet_file = BytesIO()
data_df.to_parquet(parquet_file, engine='pyarrow')
parquet_file.seek(0)

azStorageClient = AzStorageClient(az_cred_file)
azStorageClient.uploadBlob(container, blob_name, parquet_file, overwrite=True)

File uploaded to test-sites-data/test-sites-data_v_0.parquet
